## Balandis, Alina
## Gelabert Gómez, Martí


# Imports 🦄

In [5]:
!pip install scikit-fuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 993 kB 5.3 MB/s 
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894089 sha256=f79bd7e1f42e9fbc1604e181b95348c06fd7f6597313a04f04cc56701c74f3d6
  Stored in directory: /root/.cache/pip/wheels/d5/74/fc/38588a3d2e3f34f74588e6daa3aa5b0a322bd6f9420a707131
Successfully built scikit-fuzzy


In [5]:
# imports bla bla bla
import sklearn
import skfuzzy as fuzz
import numpy as np
import pandas as pd
from sklearn.datasets import *
import matplotlib.pyplot as plt
import math
import seaborn as sns

# Classic sample export
from sklearn.preprocessing import MinMaxScaler

# Functions

## Normalization and data preprocessing

In [6]:
# Normalization of your data
# now everything will be set in a range of 0 to 1
def splitLastColumn(x):
  """ Returns two numpy vector corresponding to x and y. Splits last columns, and y is transformed to column to row shape.
      [x1,x2,x3,x4,y] -> [x1,x2,x3,x4] [y]    
  """
  # the [:end] will not take the end number position in count
  end = x.shape[1]-1
  y = x[:,end:]
  return (np.array(x[:,:end]),np.array(y).flatten())

def normalize_data(x):
    """ Appling minmax normalization on the y axis sample = [x1,x2,x3,x4]"""
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(x)
    return data_scaled, scaler

# each row represents one feature from all samples in order
def reshape_feature(x):
  values = []
  for feature in range(x.shape[1]):
    f = []
    for i in range(x.shape[0]):
      f.append(x[i][feature])
    values.append(f)
  return np.array(values)



## Generating function peaks

In [7]:
# Generate an arrays with a 3 element list for each k partition
# with the values of a triangular shape functions like
# [begining, peak value, ending] 
def k_partitions(k: int, min: int, max: int) -> np.ndarray:
  """  Generate an arrays with a 3 element list for each k partition with the values of a triangular shape functions like
       [begining, peak value, ending] points """
  spaces = []
  portion = (max-min)/(k-1)
  counter = 0
  for i in range(k):
    if(i==0): # first triangle is just half of it
      spaces.append([min,min, min + portion])
      counter = min + portion
    else:
      if(i==(k-1)): # last triangle is just half of it
        spaces.append([max-portion,max, max])
      else: # General cases
        spaces.append([counter-portion,counter,counter+portion])
        counter=counter+portion
  return np.array(spaces)


## Fuzzy functions

In [39]:
def mu(x,j,k) -> np.ndarray:
  a = (j - 1)/(k-1)
  b = 1/(k-1)
  value = 1 - abs(x-a)/b
  if value <=0:
    return 0
  else:
    return value

# For each feature you will have a k lenght vector with the 
# membership degree of that place.
def membership_for_all_features(x: np.ndarray, k:np.ndarray):
  """For each feature you will have a k lenght vector with the membership degree of that place."""
  output = []
  n_features = len(k)
  for sample in range(len(x)):
    membership_i = []
    # for every feature in the sample
    i = 0
    for f in range(n_features):
      tmp=[]
      # for every subspace of that feature
      for j in range(k[i]):
        if n_features == 1:
          tmp.append(mu(x[sample],j+1,k[i]))
        else:
          tmp.append(mu(x[sample][f],j+1,k[i]))
      # check for the next k_i
      i+=1
      membership_i.append(tmp)
    # when you finish to append you will get the membership of every characteristic of that sample  
    output.append(membership_i)
  return output

#####IMPORTANT CHANGE THIS TO BE DINAMIC
# Here we just generate all posible combinations for all the feature k_i
def combinations_for_k(k:np.ndarray):
  """ Here we just generate all posible combinations for all the feature k_i 
      for example k = [2,2] will return
      [[1,1],[1,2],[2,1],[2,2]]
  """
  output = []
  labels = len(k)
  ### Change this
  if len(k)==3:
    for j in range(k[0]):
      for r in range(k[1]):
        for m in range(k[2]):
          output.append([j,r,m])
    return output
  if len(k)==2:
    for j in range(k[0]):
      for r in range(k[1]):
        output.append([j,r])
    return output
  else:
    for j in range(k[0]):
      output.append([j])
    return output

# now we need to compute every degree of compatibility with every combination
# we will have k_1*k2*...K_n diferent combinations
# if one sample is provided use [[0,3  0,2  0,3]]
def degree_of_compatibility(membership_values, combinations):
  output = []
  n_features = len(combinations[0])
  n_samples = len(membership_values)

  for i in range(n_samples):
    general = []
    for c in combinations:
      tmp = []
      for j in range(n_features):
        if len(membership_values[i])==1:
            tmp.append(membership_values[i][0][c[j]])
        else:
            tmp.append(membership_values[i][j][c[j]])
      tmp=np.array(tmp)
      general.append(np.product(tmp))
    output.append(general)
  return output

# calculating all combinations for W
def Wj1_jn(dc,k:np.ndarray, alpha=1):
    return np.power(dc,alpha)

# Now we need to compute bj1..jn, we will get a vector with the number of combinations
# posible
# Fuck x2
def bj1_jn(w, y_real):
  output = []
  n_samples=len(w)
  for j in range(len(w[0])):# these are the number of combinations we have
    numerator = [] 
    denominator = []
    for i in range(n_samples):
      numerator.append(w[i][j]*y_real[i])
      denominator.append(w[i][j])
    numerator=np.array(numerator)
    denominator=np.array(denominator)
    #print(denominator)
    value = np.sum(numerator)/np.sum(denominator)
    output.append(value)
  return output

#y(x_p)
# First version of prediction
def prediction(dc,b,w):
    output = []
    n_samples=len(w)
  
    # change samples
    for i in range(n_samples):
        numerator = [] 
        denominator = []

        for j in range(len(w[0])):# these are the number of combinations we have
            numerator.append(dc[i][j]*b[j])
            denominator.append(dc[i][j])

        numerator=np.array(numerator)
        denominator=np.array(denominator)
        #print(denominator)
        value = np.sum(numerator)/np.sum(denominator)
        output.append(value)
    return output


def execute_4_2(x,y,k,alpha,B):
  #membership_values=fuzz_membership(x,k)
  membership_values = membership_for_all_features(x,k)
  #print("Shape of membership values ->",np.array(membership_values).shape)


  # this will be the subspaces we will check on each value space
  combinations = combinations_for_k(k)
  dc = degree_of_compatibility(membership_values, combinations)
  #print("Shape of the degree of compatibility vector (samples, combinations) ->",np.array(dc).shape)

  w = Wj1_jn(dc,k,alpha)
  #print("Shape of W, should be same as dc ->",np.array(dc).shape)

  b = bj1_jn(w,y)  

  y_p,_,_=predictV2(x,b, B,k)
  #y_p = prediction(dc,b,w)

  PI = []
  for i in range(len(y)):
    a = (y_p[i]-y[i])**2
    PI.append(a/2)
    # print(f'predicted {y_p[i]} - real {y[i]}')
  PI=np.array(PI)
  print("K = ",k," alpha = ",alpha," accuracy = ", np.sum(PI) )
  return np.sum(PI),y_p,b

# This method allows to predict the values with the method shown in the section 4 of the papers
# Here we will be using the two tables
def predictV2(x,b:np.ndarray, B:np.ndarray,k:np.ndarray):

    membership_values = membership_for_all_features(b,B)# this is the good one

    membership_values_x = membership_for_all_features(x,k)
    combinations = combinations_for_k(k)

    # U_ji...jn
    dc = degree_of_compatibility(membership_values_x, combinations)
    
    # this tables only will contain the linguistic labels
    first_table = np.zeros((k[0],k[1]))
    second_table = np.zeros((k[0],k[1]))
    
    # this will contain the raw membership functions values 
    raw_first = np.zeros((k[0],k[1]))
    raw_second = np.zeros((k[0],k[1]))
    
    output_table = np.zeros((k[0],k[1]))
    
    space = k_partitions(B[0], 0, 1)
    #space = k_partitions(k, 0, 1)
   
    for i in range(len(combinations)):
        
        first_table[combinations[i][0]][combinations[i][1]]= np.argmax(membership_values[i])
        raw_first[combinations[i][0]][combinations[i][1]] = membership_values[i][0][np.argmax(membership_values[i])]

        #if combinations[i][0]==2 and combinations[i][1]==0:
        #   print(membership_values[i])
        
    for i in range(len(combinations)):    
        #####
        tmp = membership_values[i][0]
        tmp[np.argmax(membership_values[i])]=-42 # the answer to everythin is 42
        second_table[combinations[i][0]][combinations[i][1]]= np.argmax(tmp)
        
        raw_second[combinations[i][0]][combinations[i][1]]= membership_values[i][0][np.argmax(tmp)]
        
    prediction = []
    for sample in range(len(x)):
        numerator = 0
        denominator = 0
        for i in range(len(combinations)):
            # What is B**
            
            # Getting the first and the second membership space for the b fuzzy
            # in Bx[1] we will find the peak values
            B1 = space[int(first_table[combinations[i][0]][combinations[i][1]])]
            B2 = space[int(second_table[combinations[i][0]][combinations[i][1]])]
            
            numerator   += dc[sample][i]*B1[1]*raw_first[combinations[i][0]][combinations[i][1]] + dc[sample][i]*B2[1]*raw_second[combinations[i][0]][combinations[i][1]]
            denominator += dc[sample][i]*raw_first[combinations[i][0]][combinations[i][1]] + dc[sample][i]*raw_second[combinations[i][0]][combinations[i][1]]
        
        # this is literally y(p) for computing the bs
        prediction.append(numerator/denominator)
   
    return prediction, first_table, second_table


# Representations for tables

In [9]:
# we will guess that we are working with a bidimensional array of kx=5
def representation(table, combinations):
    t = ["S","MS","M","ML","L"]
    
    for c in combinations:
        print("x1 = ",t[c[0]], " x2 = ",t[c[1]]," === ", t[int(table[c[0]][c[1]])])
    
# we will guess that we are working with a bidimensional array of kx=5
def representation_num(table, combinations):
    t = ["S","MS","M","ML","L"]
    
    for c in combinations:
        print("x1 = ",t[c[0]], " x2 = ",t[c[1]]," === ", table[c[0]][c[1]])
        
def transform_table_labels(table, combinations):
    t = ["S","MS","M","ML","L"]
    a = np.array([ ['a', 'a', 'a','a','a'],
                     ['a', 'a', 'a','a','a'],
                     ['a', 'a', 'a','a','a'],
                     ['a', 'a', 'a','a','a'],
                     ['a', 'a', 'a','a','a']],dtype=object)
    for c in combinations:
        a[int(table[c[0]][c[1]])]= t[int(table[c[0]][c[1]])]
    return a
    

## Equations for testing

In [40]:
def equation13():
    """Get x and y from the paper dataset already normalized"""
    data = np.arange(0, 1.1, 0.1)
    output = []
    for x in data:
        output.append(0.2*math.sin(2*math.pi*x+math.pi/4) + 0.5)
    return data, np.array(output)

def equation25():
    """Get x and y from the paper dataset already normalized"""
    x1 = [1.40, 4.28, 1.18, 1.96, 1.85, 3.66, 3.64, 4.51, 3.77, 4.84, 1.05, 4.51, 1.84, 1.67, 2.03, 3.62, 1.67, 3.38, 2.83, 1.48, 3.37, 2.84, 1.19, 4.10, 1.65, 2.00, 2.71, 1.78, 3.61, 2.24, 1.81, 4.85, 3.41, 1.38, 2.46, 2.66, 4.44, 3.11, 4.47, 1.35, 1.24, 2.81, 1.92, 4.61, 3.04, 4.82, 2.58, 4.14, 4.35, 2.22]
    x2 = [1.80, 4.96, 4.29, 1.90, 1.43, 1.60, 2.14, 1.52, 1.45, 4.32, 2.55, 1.37, 4.43, 2.81, 1.88, 1.95, 2.23, 3.70, 1.77, 4.44, 2.13, 1.24, 1.53, 1.71, 1.38, 2.06, 4.13, 1.11, 2.27, 3.74, 3.18, 4.66, 3.88, 2.55, 2.12, 4.42, 4.71, 1.06, 3.66, 1.76, 1.41, 1.35, 4.25, 2.68, 4.97, 3.80, 1.97, 4.76, 3.90, 1.35]
    y = [3.70, 1.31, 3.35, 2.70, 3.52, 2.46, 1.95, 2.51, 2.70, 1.33, 4.63, 2.80, 1.97, 2.47, 2.66, 2.08, 2.75, 1.51, 2.40, 2.44, 1.99, 3.42, 4.99, 2.27, 3.94, 2.52, 1.58, 4.71, 1.87, 1.79, 2.20, 1.30, 1.48, 3.14, 2.22, 1.56, 1.32, 4.08, 1.42, 3.91, 5.05, 1.97, 1.92, 1.63, 1.44, 1.39, 2.29, 1.33, 1.40, 3.39]
    
    arr = np.zeros((len(x1),3))

    for i in range(len(x1)):
        arr[i][0] = x1[i]
        arr[i][1] = x2[i]
        arr[i][2] = y[i]

    data_scaled,_ = normalize_data(arr)
    x, y = splitLastColumn(data_scaled)
    return x,y

## Functions for execution

In [37]:

def execute(x,y,k=np.array([5,5]), alpha = 5, v=0, B=np.array([5])):

  membership_values = membership_for_all_features(x,k)

  # this will be the subspaces we will check on each value space
  combinations = combinations_for_k(k)
  dc = degree_of_compatibility(membership_values, combinations)
    
  w = Wj1_jn(dc,k,alpha)

  b = bj1_jn(w,y)  
    
  if v==0:
      y_p = prediction(dc,b,w)
  if v == 1:
      y_p,_,_ = predictV2(x, b, B, k)
  else:
    print("Predict with method from section 3")
    y_p = prediction(dc,b,w)
    
  PI = []
  for i in range(len(y)):
    a = (y_p[i]-y[i])**2
    PI.append(a/2)
    # print(f'predicted {y_p[i]} - real {y[i]}')
  PI=np.array(PI)
  #print("K = ",k," alpha = ",alpha," accuracy = ", np.sum(PI) )
  return np.sum(PI),y_p,b


def execute_test(x,y,ks= [[2,2],[3,3],[4,4],[5,5]], alphas = [0.1,0.5,1,5,10,50,100], B=[5], v=0):
    """This method will execute a secuence of tests for evaluation"""
    print("Prediction with section ", v+3 , " method")
    general = []
    for alpha in alphas:
      row = [alpha]
      for k in ks:
        a,out,_ = execute(x,y,np.array(k),alpha,v,B)
        row.append(a)
      general.append(row)

    # Create the pandas DataFrame
    df = pd.DataFrame(general, columns=['alpha', 'K = 2','K = 3','K = 4','K = 5'])
    # print dataframe.
    return df





In [61]:
execute(x,y,k=np.array([5,5]),alpha = 5, v=1, B=np.array([2]))[0]

0.16982369076831957

In [34]:
#x,y=equation25()
#execute_test(x,y,ks= [[2,2],[3,3],[4,4],[5,5]], alphas = [0.1,0.5,1,5,10,50,100], B=[5], v=0)
#execute_test(x,y,ks= [[2,2],[3,3],[4,4],[5,5]], alphas = [0.1,0.5,1,5,10,50,100], B=[5], v=1)

Prediction with section  3  method


,alpha,K = 2,K = 3,K = 4,K = 5
0,0.1000,1.6220,0.7466,0.5654,0.4154
1,0.5000,1.1827,0.6298,0.4620,0.3216
2,1.0000,0.9257,0.5231,0.3738,0.2490
3,5.0000,0.7276,0.4051,0.2333,0.1698
4,10.0000,0.9782,0.5662,0.2340,0.1941
5,50.0000,1.6011,0.8628,0.2614,0.2166
6,100.0000,1.6968,0.8763,0.2716,0.2170


In [31]:
B = [5]
x,y=equation25()
k = [5,5]

a,out,b = execute(x,y,np.array(k),5)




# This method returns a matrix with the b values mapped and the two tables to map linguistic labels.
def tables_and_b_table(x,b:np.ndarray, B:np.ndarray,k:np.ndarray):
    #membership_values = fuzz_membership(b,B)
    membership_values = membership_for_all_features(b,B)# this is the good one

    membership_values_x = membership_for_all_features(x,k)
    combinations = combinations_for_k(k)

    # U_ji...jn
    dc = degree_of_compatibility(membership_values_x, combinations)
    
    # this tables only will contain the linguistic labels
    first_table = np.zeros((k[0],k[1]))
    second_table = np.zeros((k[0],k[1]))
    
    # this will contain the raw membership functions values 
    raw_first = np.zeros((k[0],k[1]))
    raw_second = np.zeros((k[0],k[1]))
    
    output_table = np.zeros((k[0],k[1]))
    
    space = k_partitions(B[0], 0, 1)
    #space = k_partitions(k, 0, 1)
   
    for i in range(len(combinations)):
        
        first_table[combinations[i][0]][combinations[i][1]]= np.argmax(membership_values[i])
        raw_first[combinations[i][0]][combinations[i][1]] = membership_values[i][0][np.argmax(membership_values[i])]

        #if combinations[i][0]==2 and combinations[i][1]==0:
        #   print(membership_values[i])
        
    for i in range(len(combinations)):    
        #####
        tmp = membership_values[i][0]
        tmp[np.argmax(membership_values[i])]=-42 # the answer to everythin is 42
        second_table[combinations[i][0]][combinations[i][1]]= np.argmax(tmp)
        
        raw_second[combinations[i][0]][combinations[i][1]]= membership_values[i][0][np.argmax(tmp)]
    
    for i in range(len(combinations)):
         output_table[combinations[i][0]][combinations[i][1]]=b[i]
   
    return output_table, first_table,second_table

In [14]:
output,f,s =deriving_second(x,b,np.array(B) ,np.array(k))        
ax = sns.heatmap(output  , annot = True, cmap ='plasma',linecolor ='black', linewidths = 1)
ax.invert_yaxis()  

representation_num(output, combinations = combinations_for_k(k))
#transform_table_labels(s,combinations_for_k(k))

#ax = sns.heatmap(s  , annot = True, cmap ='plasma',linecolor ='black', linewidths = 1)
#ax.invert_yaxis() 


#representation(s,combinations_for_k(k))

#deriving(b,np.array(B),np.array(k))

NameError: name 'deriving_second' is not defined

In [15]:
def execute_4_2(x,y,k,alpha,B):
  #membership_values=fuzz_membership(x,k)
  membership_values = membership_for_all_features(x,k)
  #print("Shape of membership values ->",np.array(membership_values).shape)


  # this will be the subspaces we will check on each value space
  combinations = combinations_for_k(k)
  dc = degree_of_compatibility(membership_values, combinations)
  #print("Shape of the degree of compatibility vector (samples, combinations) ->",np.array(dc).shape)

  w = Wj1_jn(dc,k,alpha)
  #print("Shape of W, should be same as dc ->",np.array(dc).shape)

  b = bj1_jn(w,y)  

  y_p=predict_4_2(x,b, B,k)
  #y_p = prediction(dc,b,w)
  print(len(y_p))
  PI = []
  for i in range(len(y)):
    a = (y_p[i]-y[i])**2
    PI.append(a/2)
    # print(f'predicted {y_p[i]} - real {y[i]}')
  PI=np.array(PI)
  print("K = ",k," alpha = ",alpha," accuracy = ", np.sum(PI) )
  return np.sum(PI),y_p,b

B = [5]
x,y=equation25()
k = [5,5]

execute_4_2(x,y,k,0.1,B)

NameError: name 'predict_4_2' is not defined

# Exections

## equation (12)

$$y=0.2sin(2\pi x+\dfrac{\pi}{4}+0.5, \qquad 0 \leq x \leq 1)$$

In [16]:
#data = np.linspace(0.0, 1.0, num=3,dtype="float")
data = np.arange(0, 1.1, 0.1)
def test(data): #equation 13
  output = []
  for x in data:
    output.append(0.2*math.sin(2*math.pi*x+math.pi/4) + 0.5)
  return np.array(output)

y=test(data)
x = np.array(data)

ks= [[2],[3],[4],[5]]
alphas = [0.1,0.5,1,5,10,50,100]
  
execute_getIP(np.array(ks),alphas,x,y)

Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method


,alpha,K = 2,K = 3,K = 4,K = 5
0,0.1,0.107858,0.070117,0.041307,0.019521
1,0.5,0.100749,0.059839,0.031934,0.015040
2,1.0,0.095313,0.051295,0.024918,0.011344
3,5.0,0.125598,0.043903,0.014611,0.005621
4,10.0,0.164535,0.048998,0.014437,0.005394
5,50.0,0.199616,0.052222,0.015990,0.005390
6,100.0,0.199998,0.052223,0.016020,0.005390


## equation (25)

$$y=(1-x_1^{-2}+x_2^{-15})^2, \qquad 1\leq x_1,x_2 \leq 5$$

In [17]:
# TEST FOR VALUES
# THE FUNCTIONS ARE OKEY, IF THERE IS A PROBLOM SHOULD BE ON THE TABLES
x,y=equation25()

ks = [[2,2],[3,3],[4,4],[5,5]]
alphas = [0.1,0.5,1,5,10,50,100]

execute_getIP(np.array(ks),alphas,x,y)

Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method


,alpha,K = 2,K = 3,K = 4,K = 5
0,0.1,1.622001,0.746565,0.565425,0.415411
1,0.5,1.182693,0.629834,0.462009,0.321578
2,1.0,0.925733,0.523079,0.373824,0.248956
3,5.0,0.727583,0.405085,0.233279,0.169824
4,10.0,0.978236,0.566194,0.233968,0.194113
5,50.0,1.601089,0.862842,0.261424,0.216598
6,100.0,1.696847,0.876255,0.271551,0.217007


In [18]:
# >:c its gut
k_partitions(5,0, 1)
    

array([[0.  , 0.  , 0.25],
       [0.  , 0.25, 0.5 ],
       [0.25, 0.5 , 0.75],
       [0.5 , 0.75, 1.  ],
       [0.75, 1.  , 1.  ]])

# Problem Assigment

In [19]:
# Experience 	Complexity 	Quality
df = pd.read_csv("fz.csv", sep=";",names=["Experience", "Complexity", "Quality"])

data  = df.to_numpy()
data  = np.float64(data)
data_scaled,_ = normalize_data(data)

####
x, y = splitLastColumn(data_scaled)
####

ks= [[2,2],[3,3],[4,4],[5,5]]
alphas = [0.1,0.5,1,5,10,50,100]  

pd.options.display.float_format = '{:,.4f}'.format
execute_getIP(np.array(ks),alphas,x,y).to_csv('file1.csv')

Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method
Prediction with section 3 method


In [20]:
a,out,b = execute(x,y,np.array([5,5]),5)

out,first,second = deriving(b,np.array([5]) ,np.array([5,5]))

ax = sns.heatmap( first, annot = True, cmap ='plasma',linecolor ='black', linewidths = 1)
ax.invert_yaxis() 

representation(first, combinations_for_k(np.array([5,5])))


Prediction with section 3 method


NameError: name 'deriving' is not defined